# Precinct Matching Framework

In [1]:
import pandas as pd
import geopandas as gpd
from pprint import pprint

from op_verification.reference_data import (
    geoid_to_county_name,
    state_to_fips,
    state_to_state_po,
    state_abbreviation_to_state_name,
    state_fip_to_county_to_geoid,
)

### Import the datasets

In [2]:
county_id = 'Cherokee County'

county_results_filename = 'election_results_county_id={}.csv'.format(county_id)
county_results_df = pd.read_csv(county_results_filename)
county_shapefile_filename = 'shapefile_county_id={}'.format(county_id)
county_shapefile_gdf = gpd.read_file(county_shapefile_filename)

# correct for the truncation caused by 10 character column name limit in shapefiles
county_shapefile_gdf.rename(columns={'original_p':'original_precinct_name'}, inplace=True)

The next cell aliases `county_results_df` as `df` and `county_shapefile_gdf` as `gdf` here because typing fewer characters allows for faster data exploration. 

In [3]:
df = county_results_df.copy()
gdf = county_shapefile_gdf.copy()

In [4]:
df.head()

,county,precinct,Gov_DEM,Gov_REP,LtGov_DEM,LtGov_REP,StHOR_DEM,StHOR_IND,StHOR_LIB,StHOR_REP,StSen_DEM,StSen_IND,StSen_REP,SP_DEM,SP_LIB,SP_REP,USHOR_DEM,USHOR_REP,county_id,original_precinct_name
0,Cherokee County,ABSENTEE,59.0,243.0,57.0,242.0,NaN,NaN,NaN,253.0,NaN,56.0,233.0,20.0,NaN,168.0,53.0,245.0,Cherokee County,ABSENTEE
1,Cherokee County,BROOMTOWN FIRE DEPT_,14.0,238.0,17.0,234.0,NaN,NaN,NaN,231.0,NaN,34.0,217.0,9.0,NaN,166.0,17.0,234.0,Cherokee County,BROOMTOWN FIRE DEPT_
2,Cherokee County,CEDAR BLUFF 1ST BAPTIST,150.0,658.0,140.0,660.0,NaN,NaN,NaN,672.0,NaN,182.0,600.0,64.0,NaN,442.0,140.0,662.0,Cherokee County,CEDAR BLUFF 1ST BAPTIST
3,Cherokee County,CONGREGATIONAL METHODIST,22.0,108.0,22.0,105.0,NaN,NaN,NaN,101.0,NaN,28.0,95.0,15.0,NaN,57.0,25.0,101.0,Cherokee County,CONGREGATIONAL METHODIST
4,Cherokee County,DANIELS CHAPEL FELLOWSHIP,28.0,95.0,26.0,97.0,NaN,NaN,NaN,98.0,NaN,30.0,91.0,7.0,NaN,58.0,23.0,97.0,Cherokee County,DANIELS CHAPEL FELLOWSHIP


In [5]:
gdf.head()

,STATEFP,COUNTYFP,VTDST,NAMELSAD,VTDI,LSAD,CHNG_TYPE,ORIG_NAME,ORIG_CODE,RELATE,NAME,VINTAGE,FUNCSTAT,JUSTIFY,MTFCC,county_id,original_precinct_name,geometry
0,01,019,000011,Friendship Baptist Church/Mt. Calvery Baptist ...,A,00,None,None,None,None,Friendship Baptist Church/Mt. Calvery Baptist ...,90,N,None,G5240,Cherokee County,Friendship Baptist Church/Mt. Calvery Baptist ...,"POLYGON Z ((-85.62757 34.39618 0.00000, -85.62..."
1,01,019,000021,Broomtown Volunteer Fire Dept.,A,00,None,None,None,None,Broomtown Volunteer Fire Dept.,90,N,None,G5240,Cherokee County,Broomtown Volunteer Fire Dept.,"POLYGON Z ((-85.61464 34.36811 0.00000, -85.61..."
2,01,019,000031,Valley Church,A,00,None,None,None,None,Valley Church,90,N,None,G5240,Cherokee County,Valley Church,"POLYGON Z ((-85.69221 34.29488 0.00000, -85.69..."
3,01,019,000041,Unity Missionary Baptist Church,A,00,None,None,None,None,Unity Missionary Baptist Church,90,N,None,G5240,Cherokee County,Unity Missionary Baptist Church,"POLYGON Z ((-85.67004 34.24717 0.00000, -85.66..."
4,01,019,000051,Gaylesville Town Hall,A,00,None,None,None,None,Gaylesville Town Hall,90,N,None,G5240,Cherokee County,Gaylesville Town Hall,"POLYGON Z ((-85.61212 34.26803 0.00000, -85.60..."


### Check Preconditions
These should all pass - they're here to ensure that everything in `setup.ipynb` worked correctly

In [6]:
# TODO: Pass the precondition described above which takes the form of an assert statement in this cell.
assert 'county_id' in df.columns and 'county_id' in gdf.columns
assert 'original_precinct_name' in df.columns and 'original_precinct_name' in gdf.columns

### General Modifications
Its normally benificial to apply some modifications uniformly to all precincts. For example, its good practice to make everything lower case. This modification is made in `edit_precinct_name` - read its specification to learn more about how to use it to make more modifications. Consider removing substrings that appear in every precinct name like `voting district`. Be careful of removing words that will result in duplicate precinct names. For example, if there are two `Newtown Voting District` precincts in the shapefile, and the election results have `Newtown Boro` and `Newtown Township`, its okay to remove `Voting District`, but you probably don't want to remove `Boro` or `Township`. Of course, this will differ from County to County - so be vigilant!

In [8]:
dataset_name_df = 'df'
dataset_name_gdf = 'gdf'

def edit_precinct_name(prec_name, 
    remove_lst=[], 
    target_to_replacement={},
    stopping_words=[],
    prec_dict={}):
    '''
    Returns a lower case precinct name (string) with certian modifications depending other arguments. 
    
    Modifications are performed in order of the parameters they depend on. By convention, case is 
    ignored by making prec_name lower case. Accordingly, one should pass arguements with lower case
    elements. That is, keys of the dictionaries and elements of lists should be lower case strings.

	Parameters:
		prec_name (str): precinct name
		remove_lst ((str) list): if a string in this list is a substring in prec_name it will be removed. 
            All elements should be lower case.
        target_to_replacement ({str:str} dictionary): keys (targets) will be replaced with their 
            corresponding value (replacements) in prec_name. All keys should be lower case.
        stopping_words ({str} list): If any substring of prec_name contains a element of stopping_words
             that is adjacent to a space character it will be removed. All elements should be lower case.
        prec_dict ({str:str} dictionary): After all the modifications above, if the edited prec_name
            string is in the set of keys for prec_dict, then it will be replaced with that key's value.  
            All keys should be lower case.

	Returns:
		prec_name (str): prec_name arguement returned with the 
    '''
    prec_name = str(prec_name).lower()
    for word in remove_lst:
        prec_name = prec_name.replace(word, '')
    for target, replacement in target_to_replacement.items():
        prec_name = prec_name.replace(target, replacement)
    words = prec_name.split()
    words = [word.lstrip('0') for word in words if word not in stopping_words]
    prec_name = " ".join(words)
    return prec_dict[prec_name] if prec_name in prec_dict.keys() else prec_name

df_to_gdf = {
 'broomtown fire dept_': 'broomtown volunteer fire dept.',
 'cedar bluff 1st baptist': 'cedar bluff first baptist church',
 'daniels chapel fellowship': "daniel's chapel (little river )",
 'congregational methodist': 'ebenezer methodist church',
 'friendship bap fellowship': 'friendship baptist church/mt. calvery baptist church',
 'gaylesville city hall': 'gaylesville town hall',
 'goshen methodist church': 'goshen united methodist church',
 'leesburg fema bldg_': 'leesburg volunteer fire dept.',
 'mccords crossroads fire': "mccord's crossroad volunteer fire dept./ rock run church",
}

gdf_to_df = {
}

# Tune the matching by adding optional arguements to edit
df['edited_precinct_name'] = df['original_precinct_name'].apply(lambda name: edit_precinct_name(name,prec_dict=df_to_gdf))
gdf['edited_precinct_name'] = gdf['original_precinct_name'].apply(lambda name: edit_precinct_name(name,prec_dict=gdf_to_df))

######## Manual Corrections ###########
# Make precinct specific corrections here like splitting one precinct into two because of new congressional districts
# e.g gdf.loc[gdf['VTDST']=='000290','edited_precinct_name'] = 'howard township'

# make the 'original_precinct_name, edited_precinct_name' for use in the loop below
df['original_precinct_name, edited_precinct_name'] = df[['original_precinct_name','edited_precinct_name']].apply(tuple, axis=1)
gdf['original_precinct_name, edited_precinct_name'] = gdf[['original_precinct_name','edited_precinct_name']].apply(tuple, axis=1)

######## Matching Framework ###########
unmatched_precinct_lst_gdf = []
unmatched_precinct_lst_df = ['absentee', 'provisional']

precinct_list_df = sorted(list(df[df['county_id'] == county_id]['original_precinct_name, edited_precinct_name'].unique()), key=lambda x: x[1])
precinct_list_gdf = sorted(list(gdf[gdf['county_id'] == county_id]['original_precinct_name, edited_precinct_name'].unique()), key=lambda x: x[1])

precinct_set_df = {x[1] for x in precinct_list_df if x[1] not in unmatched_precinct_lst_df}
precinct_set_gdf = {x[1] for x in precinct_list_gdf if x[1] not in unmatched_precinct_lst_gdf}

unmatched_precincts_df = sorted(list(precinct_set_df - precinct_set_gdf))
unmatched_precincts_gdf = sorted(list(precinct_set_gdf - precinct_set_df))
n_unmatched = len(unmatched_precincts_df) + len(unmatched_precincts_gdf)
if n_unmatched > 0:
        print("county_id: '{}' | {} precincts in {} | {} precincts in {}:\n".format(county_id, len(precinct_list_df), dataset_name_df, len(precinct_list_gdf), dataset_name_gdf))
        n_precincts_total = len(precinct_list_df) + len(precinct_list_gdf)
        print(n_unmatched, " precincts are unmatched out of ", n_precincts_total)
        df_unmatched = df[(df['edited_precinct_name'].isin(unmatched_precincts_df)) & (df.county_id == county_id)]
        gdf_unmatched = gdf[(gdf['edited_precinct_name'].isin(unmatched_precincts_gdf)) & (gdf.county_id == county_id)]
        if n_unmatched > 100:
            print("\nLook for parterns and use change the parameters to edit_precinct_name accordingly.\n")
            for index, (original_precinct_name_df, edited_precinct_name_df) in enumerate(precinct_list_df):
                original_precinct_name_gdf, edited_precinct_name_gdf = precinct_list_gdf[index]
                if edited_precinct_name_df in unmatched_precincts_df and edited_precinct_name_gdf in unmatched_precincts_gdf:
                    print("{} <-- {} ({})".format(edited_precinct_name_df, original_precinct_name_df, dataset_name_df))
                    print("{} <-- {} ({})\n".format(edited_precinct_name_gdf, original_precinct_name_gdf, dataset_name_gdf))
        else:
            print("unmatched_precincts_df ({}) - len = {}| '{}':".format(dataset_name_df, len(unmatched_precincts_df), county_id), unmatched_precincts_df)
            print("\nunmatched_precincts_gdf ({}) - len = {}| '{}':".format(dataset_name_gdf, len(unmatched_precincts_gdf), county_id), unmatched_precincts_gdf)
            precinct_modification_dictionary_df_to_gdf = {unmatched_precincts_df[i]: unmatched_precincts_gdf[i] if i < len(unmatched_precincts_gdf) else '' for i in range(len(unmatched_precincts_df))}
            precinct_modification_dicitonary_gdf_to_df = {unmatched_precincts_gdf[i]: unmatched_precincts_df[i] if i < len(unmatched_precincts_df) else '' for i in range(len(unmatched_precincts_gdf))}
            print("{}  to {} precinct modification dictionary: ".format(dataset_name_df, dataset_name_gdf))
            print("'{}':".format(county_id))
            pprint(precinct_modification_dictionary_df_to_gdf)
            print("{}  to {} precinct modification dictionary: ".format(dataset_name_gdf, dataset_name_df))
            print("'{}':".format(county_id))
            pprint(precinct_modification_dicitonary_gdf_to_df)
            for index, (original_precinct_name_df, edited_precinct_name_df) in enumerate(precinct_list_df):
                original_precinct_name_gdf, edited_precinct_name_gdf = precinct_list_gdf[index]
                if edited_precinct_name_df in unmatched_precincts_df or edited_precinct_name_gdf in unmatched_precincts_gdf:
                    print("{} <-- {} ({})".format(edited_precinct_name_df, original_precinct_name_df, dataset_name_df))
                    print("{} <-- {} ({})\n".format(edited_precinct_name_gdf, original_precinct_name_gdf, dataset_name_gdf))
            else:
                print("Add unmatched precincts to the unmatched precinct.")
else:
    print("All Done! (make sure you have one to one matches)")

county_id: 'Cherokee County' | 25 precincts in df | 20 precincts in gdf:

23  precincts are unmatched out of  45
unmatched_precincts_df (df) - len = 13| 'Cherokee County': ['ebenezer fellowship hall', 'mt calvary baptist church', 'mt_ weisner fire hall', 'mt_ wesiner vfd #2', 'new hope methodist', 'rock run church', 'shady grove church', 'spring creek vfd', 'spring garden fire sta_', 'the roc', 'the valley church', "tucker's chapel fire dept", 'unity missionary baptist']

unmatched_precincts_gdf (gdf) - len = 10| 'Cherokee County': ['ellisville volunteer fire dept.', 'mt. weisner volunteer fire dept./melrose baptist church', 'new hope methodist church', 'recreational outreach center', 'shady grove fellowship hall', 'spring creek volunteer fire dept.', 'spring garden volunteer fire dept.', "tucker's chapel vfw", 'unity missionary baptist church', 'valley church']
df  to gdf precinct modification dictionary: 
'Cherokee County':
{'ebenezer fellowship hall': 'ellisville volunteer fire dept

IndexError: list index out of range

### Validate

In [8]:
acceptable_duplicates_to_reason_df = {
}
counts = df['edited_precinct_name'].value_counts()
duplicates = counts[(counts>1) & (~counts.index.isin(acceptable_duplicates_to_reason_df.keys()))]
assert len(duplicates) == 0

In [9]:
acceptable_duplicates_to_reason_gdf = {
}
counts = gdf['edited_precinct_name'].value_counts()
duplicates = counts[(counts>1) & (~counts.index.isin(acceptable_duplicates_to_reason_gdf.keys()))]
assert len(duplicates) == 0

In [10]:
gdf.edited_precinct_name.nunique()

49

In [11]:
gdf.original_precinct_name.nunique()

49

In [12]:
df.edited_precinct_name.nunique()

51

In [13]:
df.edited_precinct_name.nunique()

51

In [14]:
df = df[~df['edited_precinct_name'].isin(unmatched_precinct_lst_df)]

### Export

In [17]:
df['edited_precinct_name'] = df['edited_precinct_name'].str.title()
df['loc, prec'] = df.apply(lambda row: county_id + ', ' + row['edited_precinct_name'], axis=1)
df = df.groupby(by='loc, prec').sum()

In [18]:
gdf['edited_precinct_name'] = gdf['edited_precinct_name'].str.title()
gdf['loc, prec'] = gdf.apply(lambda row: county_id + ', ' + row['edited_precinct_name'], axis=1)
gdf = gdf.dissolve(by='loc, prec', as_index=False)
county_to_geoid = state_fip_to_county_to_geoid[1]
gdf['GEOID'] = gdf['county_id'].map(county_to_geoid)
gdf = gdf.set_index('loc, prec')

In [20]:
cols = [ 'loc, prec', 'GEOID', 'county_id', 'edited_precinct_name'] + list(df.columns) + ['geometry']
cols

['loc, prec',
 'GEOID',
 'county_id',
 'edited_precinct_name',
 'Gov_DEM',
 'Gov_REP',
 'LtGov_DEM',
 'LtGov_REP',
 'StHOR_DEM',
 'StHOR_IND',
 'StHOR_LIB',
 'StHOR_REP',
 'StSen_DEM',
 'StSen_IND',
 'StSen_REP',
 'SP_DEM',
 'SP_LIB',
 'SP_REP',
 'USHOR_DEM',
 'USHOR_REP',
 'geometry']

In [21]:
joined_df = df.join(gdf, lsuffix='_left', rsuffix='_right').reset_index()
print(joined_df.shape)
joined_df.head(2)

(49, 38)


,"loc, prec",Gov_DEM,Gov_REP,LtGov_DEM,LtGov_REP,StHOR_DEM,StHOR_IND,StHOR_LIB,StHOR_REP,StSen_DEM,...,NAME,VINTAGE,FUNCSTAT,JUSTIFY,MTFCC,county_id,original_precinct_name,edited_precinct_name,"original_precinct_name, edited_precinct_name",GEOID
0,"Baldwin County, Barnwell Vfd",418.0,1621.0,350.0,1670.0,421.0,0.0,0.0,1605.0,365.0,...,Barnwell VFD,90,N,None,G5240,Baldwin County,Barnwell VFD,Barnwell Vfd,"(Barnwell VFD, barnwell vfd)",01003
1,"Baldwin County, Bay Minette City Hall / Civic Ctr",641.0,2046.0,622.0,2044.0,645.0,0.0,0.0,1998.0,0.0,...,Bay Minette City Hall / Civic Ctr,90,N,None,G5240,Baldwin County,Bay Minette City Hall / Civic Ctr,Bay Minette City Hall / Civic Ctr,"(Bay Minette City Hall / Civic Ctr, bay minett...",01003


In [22]:
output = gpd.GeoDataFrame(joined_df[cols].rename(columns={'edited_precinct_name':'precinct'}))
output.head()

,"loc, prec",GEOID,county_id,precinct,Gov_DEM,Gov_REP,LtGov_DEM,LtGov_REP,StHOR_DEM,StHOR_IND,...,StHOR_REP,StSen_DEM,StSen_IND,StSen_REP,SP_DEM,SP_LIB,SP_REP,USHOR_DEM,USHOR_REP,geometry
0,"Baldwin County, Barnwell Vfd",01003,Baldwin County,Barnwell Vfd,418.0,1621.0,350.0,1670.0,421.0,0.0,...,1605.0,365.0,0.0,1638.0,191.0,0.0,1104.0,344.0,1690.0,"POLYGON Z ((-88.01621 30.40448 0.00000, -88.01..."
1,"Baldwin County, Bay Minette City Hall / Civic Ctr",01003,Baldwin County,Bay Minette City Hall / Civic Ctr,641.0,2046.0,622.0,2044.0,645.0,0.0,...,1998.0,0.0,0.0,2116.0,396.0,0.0,1335.0,597.0,2085.0,"POLYGON Z ((-87.92361 30.93146 0.00000, -87.92..."
2,"Baldwin County, Belforest Community Ctr",01003,Baldwin County,Belforest Community Ctr,399.0,1453.0,346.0,1497.0,345.0,0.0,...,1433.0,346.0,0.0,1481.0,158.0,4.0,964.0,327.0,1521.0,"POLYGON Z ((-87.88146 30.55826 0.00000, -87.88..."
3,"Baldwin County, Bethel Bapt Church",01003,Baldwin County,Bethel Bapt Church,78.0,588.0,63.0,599.0,69.0,0.0,...,590.0,0.0,0.0,605.0,33.0,0.0,414.0,63.0,602.0,"POLYGON Z ((-87.67327 30.56642 0.00000, -87.67..."
4,"Baldwin County, Bon Secour Morgan'S Chapel Um ...",01003,Baldwin County,Bon Secour Morgan'S Chapel Um Church,233.0,898.0,221.0,910.0,0.0,0.0,...,936.0,211.0,0.0,912.0,126.0,0.0,670.0,214.0,915.0,"POLYGON Z ((-88.02759 30.30582 0.00000, -88.02..."


In [23]:
parent_dir = '../../data/shapefiles/county_level_precinct_election_results'
output.to_file('/'.join([parent_dir, "_".join(county_id.split())]))